<a href="https://colab.research.google.com/github/prashantiyaramareddy/AI-ML-Learnings/blob/master/ComputerVision/VGGNet_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

VGG-Net Implementation

VGGNet is a convolutional neural network architecture proposed by the Visual Geometry Group (VGG) from Oxford University. It is known for its simplicity and uniform architecture, using small convolutional filters and deep layers.

In [ ]:
### Import Required Libraries
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models
from torch.optim.lr_scheduler import ReduceLROnPlateau


In [ ]:
torch.__version__

In [ ]:
# Check GPU Availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

### Define transforms to be applied on dataset

In [ ]:
from torch.nn.modules import padding

# Use the mean and sta from CIFAR10
mean = (0.4914, 0.4822, 0.4465)
std = (0.2023, 0.1994, 0.2010)

train_transform = torchvision.transforms.Compose([
    torchvision.transforms.RandomCrop(32, padding=4),
    torchvision.transforms.RandomHorizontalFlip(p=0.5),
    transforms.Resize((224, 224)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean, std)
])

test_transform = torchvision.transforms.Compose([
    transforms.Resize((224, 224)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean, std)
])


In [ ]:
# Import Dataset
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)



In [ ]:
# Create DataLoader
val_size = 5000
train_size = len(train_dataset) - val_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, pin_memory=True)

In [ ]:
len(train_loader), len(test_loader), len(val_loader)

In [ ]:
### Load Pretrained Model
vggnet = models.vgg19(pretrained=True)

# Freeze all layers
for param in vggnet.parameters():
    param.requires_grad = False

# Replace the last fully connected layer
num_classes = 10
vggnet.classifier[6] = nn.Linear(4096, num_classes)

# Only the new final layer should be trainable
for param in vggnet.classifier[6].parameters():
    param.requires_grad = True

vggnet = vggnet.to(device)

In [ ]:
# Train the Model
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(vggnet.parameters(), lr=0.001,
                       weight_decay=5e-4)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5)

model = vggnet

In [ ]:
from google.colab import output

epochs = 20

for epoch in range(epochs):
  model.train()
  total_EpochLoss = 0
  for images, labels in train_loader:
    images = images.to(device)
    labels = labels.to(device)

    # Forward pass
    outputs = model(images)

    # Calculate loss
    loss = criterion(outputs, labels)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    total_EpochLoss += loss.item()

avg_loss = total_EpochLoss / len(train_loader)
print(f"Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}")

model.eval()
val_loss = 0.0

with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)
        val_loss += loss.item()

avg_val_loss = val_loss / len(val_loader)
print(f"Validation Loss: {avg_val_loss:.4f}")

print(f"Epoch [{epoch+1}/{epochs}], Validation Loss: {avg_val_loss:.4f}")

#Step the scheduler with validation loss
scheduler.step(avg_val_loss)

# Save the checkpoint every 10 epochs
if (epoch + 1) % 10 == 0:
  checkpoint_path = f'checkpoint_epoch_{epoch + 1}.pth'
  torch.save({
      'epoch': epoch + 1,
      'model_state_dict': model.state_dict(),
      'optimizer_state_dict': optimizer.state_dict(),
      'loss': avg_loss,
  }, checkpoint_path)
  print(f"Checkpoint saved at {checkpoint_path}")




In [ ]:
model.eval()

### Evaluation in test dataset

In [ ]:
total = 0
correct = 0

with torch.no_grad():
  for batchFeatures, batchLabels in test_loader:
    # move to gpu
    batchFeatures, batchLabels = batchFeatures.to(device), batchLabels.to(device)

    # Forward pass
    outputs = model(batchFeatures)

    # Calculate Loss
    _, predicted = torch.max(outputs.data, 1)
    total += batchLabels.size(0)
    correct += (predicted == batchLabels).sum().item()

accuracy = correct / total
print(f"Test Accuracy: {accuracy * 100:.2f}%")


### Predictions

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from random import sample

# Define CIFAR-10 class names
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# Set model to evaluation mode
model.eval()

# Disable gradients for prediction
with torch.no_grad():
  for batchFeatures, batchLabels in test_loader:
    # Move to GPU
    batchFeatures, batchLabels = batchFeatures.to(device), batchLabels.to(device)

    # Forward pass
    outputs = model(batchFeatures)

    _, predicted_indices = torch.max(outputs.data, 1)

    # Select N random images
    N = 5
    indices = sample(range(len(batchFeatures)), N)

    # Plot the images using the axes object
    fig, axes = plt.subplots(1, N, figsize=(15, 5))

    # Plot images with actual and predicted labels
    for i in range(N):
      idx = indices[i]

      # Get image, label and predicted labels
      img = batchFeatures[idx].cpu()
      actual = classes[batchLabels[idx].item()]
      predicted_label = classes[predicted_indices[idx].item()]

      # Denormalize image for diaplay
      img = img * torch.tensor(std).view(3, 1, 1) + torch.tensor(mean).view(3, 1, 1)
      img = img.permute(1,2,0)
      img = torch.clamp(img, 0, 1)

      # Plot image on the respective subplot axis
      axes[i].imshow(img)
      axes[i].set_title(f"Actual: {actual}\nPredicted: {predicted_label}")
      axes[i].axis('off')

    plt.tight_layout()
    plt.show()
    break